## Sex differences in Autism Spectrum Disorder, a Comorbidity Pattern Analysis in National Scale Data: replication in Boston Children Hospital data

This analysis was run using R Version 3.6.

### Install and Load libraries

In [ ]:
install.packages( "knitr", repos='http://cran.us.r-project.org')
install.packages( "dplyr", repos='http://cran.us.r-project.org')
install.packages("tidyr", repos='http://cran.us.r-project.org')
library( "knitr" )
library( "dplyr" )
library( "tidyr" )

### ASD patient selection

In [ ]:
# Define ASD ICD9 coodes
autismIcd9codes <- c( "299.0", "299.00","299.01", "299.8", "299.80", "299.81", "299.9", "299.90", "299.91")

# Load the data extracted from BCH
load("bchRawData.Rdata")

Select those patients with at least 3 ASD diganosis.

In [ ]:
asdBch3diag <- bchRawData[ bchRawData$code %in% autismIcd9codes, c( "PATIENT_NUM", "code" ) ]
asdBch3diag <- as.data.frame( table( asdBch3diag$PATIENT_NUM ) )
asdBch3diag <- asdBch3diag[ asdBch3diag$Freq >= 3, ]

ASD <- bchRawData[ bchRawData$PATIENT_NUM %in% asdBch3diag$Var1, ]
ASD <- ASD[! ASD$code %in% autismIcd9codes, ]

##### Remove those patients that could be in Aetna
To guarante that the two sets are completely independent we remove those patients from BCH taht could be potentially in Aetna. To do this we look for those patients that had "AETNA" string in one of the fields and that had and Autism ICD9-CM code. We create a list with all the patients in BCH meeting this criteria. 

In [ ]:
# SELECT CONCEPT_CD FROM ASD_GI.CONCEPT_DIMENSION where NAME_CHAR like '%AETNA%';
# SELECT distinct PATIENT_NUM FROM ASD_GI.OBSERVATION_FACT 
# WHERE CONCEPT_CD IN  ('INS:4325','INS:2','INS:3','INS:7070','INS:4100')
# INTERSECT
# SELECT distinct PATIENT_NUM FROM ASD_GI.OBSERVATION_FACT 
# WHERE CONCEPT_CD IN ('ICD9:299.0','ICD9:299.01','ICD9:299.00','ICD9:299.80','ICD9:299.8','ICD9:299.81','ICD9:299.9','ICD9:299.90','ICD9:299.91');

Then, we remove the potential Aetna patients from BCH. 

In [ ]:
bchAetnaPatients <- read.delim("bchPatientsToAetna.csv", header = TRUE)
ASD <- ASD[! ASD$PATIENT_NUM %in% bchAetnaPatients$PATIENT_NUM, ]

#### Generate the four data groups:

##### ASD females

In [ ]:
femaleASD <- ASD[ ASD$SEX_CD == "F", c( "PATIENT_NUM", "code","START_DATE", "SEX_CD", "BIRTH_DATE", "age" ) ]
colnames( femaleASD ) <- c( "MemberId", "Icd","DateServiceStarted", "Gender", "BirthYear", "age" )
femaleASD$Caco <- "Female"

##### ASD males

In [ ]:
maleASD   <- ASD[ ASD$SEX_CD == "M",  c( "PATIENT_NUM", "code", "START_DATE", "SEX_CD", "BIRTH_DATE", "age" ) ]
colnames( maleASD ) <- c( "MemberId", "Icd","DateServiceStarted", "Gender", "BirthYear", "age" )
maleASD$Caco <- "Male"

##### Females without ASD

In [ ]:
load("noAsDBch.RData")
femaleNonASD <- DataF[ DataF$SEX_CD == "F", c( "PATIENT_NUM", "CONCEPT_CD","START_DATE", "SEX_CD", "BIRTH_DATE", "Age" )]
femaleNonASD$CONCEPT_CD <- sapply( strsplit( as.character( femaleNonASD$CONCEPT_CD), "[:]"), '[', 2 )
colnames( femaleNonASD ) <- c( "MemberId", "Icd","DateServiceStarted", "Gender", "BirthYear", "age" )
femaleNonASD$Caco <- "FemaleNonASD"

##### Males without ASD

In [ ]:
maleNonASD <- DataF[ DataF$SEX_CD == "M", c( "PATIENT_NUM", "CONCEPT_CD","START_DATE", "SEX_CD", "BIRTH_DATE", "Age" ) ]
maleNonASD$CONCEPT_CD <- sapply( strsplit( as.character( maleNonASD$CONCEPT_CD ), "[:]"), '[', 2 )
colnames( maleNonASD ) <- c( "MemberId", "Icd","DateServiceStarted", "Gender", "BirthYear", "age" )
maleNonASD$Caco <- "MaleNonASD"

### Create a function to run the comorbidity analysis
Create a function that giving as input:
- Two patient groups to compare
- The age range under study
- The minimum number of diagnosis to consider a patient having a diagnosis

Generates as an output a table with the:
- Phenotypes
- Number of patients of each group having the PheCodes
- Number of patients of each group not having the PheCodes 
- Number of patients of each group excluded for each PheCodes because they have been diagnosed but less than the minimum times

In [ ]:
myComorbidityAnalysis <- function( caseGroup, controlGroup, myDataSet, minAge = 0, maxAge= 18, 
                                  caseSymbol = "ASD", controlSymbol = "nonASD", minimunDiagnosis = 3){
    
    if( missing( myDataSet ) ){
        myCompleteData <- rbind( caseGroup, controlGroup )
    }else{
        myCompleteData <- myDataSet
    }
  
    #select diagnostics in the age range
    myCompleteData <- myCompleteData[ myCompleteData$age >= minAge &
                                      myCompleteData$age <= maxAge, ]
    
    #remove males with females ASD diagnosis
    message( "Remove males with females ASD diagnosis ..." )
    sexSpecificPheCodes <- read.csv( "phecode_definitions1.2.csv", header =TRUE, colClasses = "character" )
    femaleSpecificPheCodes <- sexSpecificPheCodes[ sexSpecificPheCodes$sex == "Female", ]
    maleSpecificPheCodes <- sexSpecificPheCodes[ sexSpecificPheCodes$sex == "Male", ]
    
    
    if(! "Phenotype" %in% colnames( myCompleteData) ){
        phemapFile <- read.csv("phecode_icd9_rolled.csv", header =TRUE, colClasses = "character")
        phemapFile <- phemapFile[, c( "ICD9", "PheCode") ]
        colnames( phemapFile ) <- c("Icd", "Phenotype")
        
        myCompleteDataFinal <- inner_join( myCompleteData, phemapFile, by = "Icd")
        rm( myCompleteData )
    }else{
        myCompleteDataFinal <- myCompleteData
        rm( myCompleteData )
    }
    
    #remove females with males ASD diagnosis and viceversa
    message("Remove females with males ASD diagnosis ...")
    femaleDiagnosisError <- myCompleteDataFinal[ myCompleteDataFinal$Phenotype %in% maleSpecificPheCodes$jd_code & 
                                                 myCompleteDataFinal$Gender == "F", ]
    myCompleteDataFinal <- myCompleteDataFinal[! myCompleteDataFinal$MemberId %in% femaleDiagnosisError$MemberId, ]
    
    
    # Case-control file: id, case or control 
    message("Generating the case-control file containing patient identifier and case/control status ...")
    caco <- myCompleteDataFinal[, c( "MemberId", "Caco" ) ]
    caco <- caco[ ! duplicated( caco ) , ]
    caco$Caco <- gsub( controlSymbol, 0, caco$Caco )
    caco$Caco <- gsub( caseSymbol, 1, caco$Caco )
    
    message( paste0( "Total number of patients in the data set", length( unique( caco$patientId ) ) ) )
    message( paste0( "Summary of controls and cases in the data set", summary( as.factor( caco$Caco ) ) ) )
    
    # Create phenotype dataframe: id, PheCode and counts #
    message("Generating the file containing patient identifier, PheCode code and counts ...")
    phenotype <- myCompleteDataFinal[, c( "MemberId", "Phenotype" ) ]
    colnames( phenotype ) <- c( "patientId", "phecode" )
    
    phenotype$pair <- paste( phenotype$patientId, phenotype$phecode, sep = "*" )
    counts <- as.data.frame( table( phenotype$pair ) )
    colnames( counts ) <- c("pair", "counts")
    
    phenotypeFinal <- inner_join( phenotype, counts, by = "pair" )
    phenotypeFinal <- phenotypeFinal[! duplicated(phenotypeFinal), c("patientId", "phecode", "counts")]
    
    # Merge with the Case-control information 
    data4f <- inner_join( phenotypeFinal, caco, by = "patientId" )
    caco$Caco <- as.character( caco$Caco )
    
    totalPhewasCodes <- unique( phenotypeFinal$phecode )
    phewasOutput     <- as.data.frame( matrix( ncol=7, nrow=length( totalPhewasCodes ) ) )
    colnames( phewasOutput ) <- c( "phecode", "caseYes", "caseNo", "caseExclude", 
                                     "controlYes", "controlNo", "controlExclude" )
    phewasOutput$phecode <- as.character( totalPhewasCodes )
    
    
    for( i in 1:length( totalPhewasCodes ) ){
        
        excludedPatients <- sum( is.na( data4f[, names( phenotypesUnderStudy )[ i ] ] ) )
        cacodf <-  data4f[, c( names( phenotypesUnderStudy )[ i ], "patientId", "Caco" ) ]
        cacodf <- na.omit( cacodf )
        
        caseDisease   <- length( unique(cacodf[ cacodf[, 1] == 1 & cacodf$Caco == "1", "patientId"]))
        caseNoDisease <- length( unique(cacodf[ cacodf[, 1] == 0 & cacodf$Caco == "1", "patientId"]))
        
        controlDisease   <- length(unique(cacodf[ cacodf[, 1] == 1 & cacodf$Caco == "0", "patientId"]))
        controlNoDisease <- length(unique(cacodf[ cacodf[, 1] == 0 & cacodf$Caco == "0", "patientId"]))
        
        newRow <- c ( names(phenotypesUnderStudy)[i], caseDisease, caseNoDisease, controlDisease, controlNoDisease )
        phewasOutput <- rbind( newRow, phewasOutput )
  }
    
    colnames( phewasOutput ) <- c( "phecode", "casePhenotype_present", "casePhenotype_absent", 
                                "controlPhenotype_present", "controlPhenotype_absent", "Excluded_patients" )
  
    message( "Done! Congratulations! Your comorbidity analysis has been finished!" )
    return( phewasOutput )
}


#Function to perform the fisher test
get_fisher <- function( df ){
    
  mat <- matrix( as.numeric( df[ c( 2,3,4,5 ) ] ), ncol=2 )
  f <- fisher.test( mat, alt = "two.sided" )
  return( c( unlist( df[ 1 ] ), f$p.value, f$conf.int, f$estimate ) )
    
}

### Apply the function to run the comorbidity analysis 
We will apply the previous function to a specific age range for the three different groups we want to compare:
- ASD females vs ASD males
- ASD females vs females without ASD
- Females without ASD vs males without ASD

In [ ]:
# ASD females vs ASD males #
maleASD$Caco   <- "male"
femaleASD$Caco <- "female"

#apply the myComorbidityAnalysis function 
maleVsFemale12to18 <- myComorbidityAnalysis( femaleAsdSet  = femaleASD, 
                                        femaleNonAsdSet  = maleASD, 
                                        minAge           = 12,
                                        maxAge           = 18,
                                        caseSymbol       = "female", 
                                        controlSymbol    = "male", 
                                        minimunDiagnosis = 3 )


maleVsFemale12to18 <- maleVsFemale12to18[ complete.cases( maleVsFemale12to18 ) , ]

#apply the fisher test function
fishers <- t( apply( maleVsFemale12to18, 1,  get_fisher ) )
colnames( fishers ) <- c("phecode", "pValue", "confIntL", "confIntH", "OR" )
maleVsFemale12to18 <- merge( maleVsFemale12to18, fishers, by = "phecode" )

#add the adjusted p-value
maleVsFemale12to18$pAdjust <- p.adjust( as.numeric( as.character( maleVsFemale12to18$pValue ) ), method = "bonferroni" )

#save the results
save( maleVsFemale12to18, file = "maleVsFemale12to18.RData" )

In [ ]:
# ASD females vs Non-ASD females #
femaleNonASD$Caco <- "nonASD"
femaleASD$Caco    <- "ASD"

#apply the myComorbidityAnalysis function 
asdVSnonAS12to18  <- myComorbidityAnalysis( femaleAsdSet = femaleASD, 
                                        femaleNonAsdSet  = femaleNonASD, 
                                        caseSymbol       = "ASD", 
                                        controlSymbol    = "nonASD", 
                                        minAge           = 12,
                                        maxAge           = 18,
                                        minimunDiagnosis = 3 )

asdVSnonAS12to18 <- asdVSnonAS12to18[ complete.cases( asdVSnonAS12to18 ), ]

#apply the fisher test function
fishers <- t( apply( asdVSnonAS12to18, 1,  get_fisher ) )
colnames( fishers ) <- c( "phecode", "pValue", "confIntL", "confIntH", "OR" )
asdVSnonAS12to18 <- merge( asdVSnonAS12to18, fishers, by = "phecode")

#add the adjusted p-value
asdVSnonAS12to18$pAdjust <- p.adjust( as.numeric( as.character( asdVSnonAS12to18$pValue ) ), method = "bonferroni" )

#save the results
save( asdVSnonAS12to18, file = "asdVSnonAS12to18.RData" )

In [ ]:
# non-ASD females vs non-ASD males #
maleNonASD$Caco   <- "male"
femaleNonASD$Caco <- "female"

#apply the myComorbidityAnalysis function 
noasdmaleVsFemale12to18 <- myComorbidityAnalysis( femaleAsdSet = femaleNonASD, 
                                        femaleNonAsdSet  = maleNonASD, 
                                        minAge           = 12,
                                        maxAge           = 18,
                                        caseSymbol       = "female", 
                                        controlSymbol    = "male", 
                                        minimunDiagnosis = 3 )

noasdmaleVsFemale12to18 <- noasdmaleVsFemale12to18[ complete.cases( noasdmaleVsFemale12to18 ), ]

#apply the fisher test function
fishers <- t( apply( noasdmaleVsFemale12to18, 1,  get_fisher ) )
colnames(fishers) <- c( "phecode", "pValue", "confIntL", "confIntH", "OR" )
noasdmaleVsFemale12to18 <- merge( noasdmaleVsFemale12to18, fishers, by = "phecode" )

#add the adjusted p-value
noasdmaleVsFemale12to18$pAdjust <- p.adjust( as.numeric( as.character( noasdmaleVsFemale12to18$pValue ) ), 
                                            method = "bonferroni" )

#save the results
save( noasdmaleVsFemale12to18, file = "noasdmaleVsFemale12to18.RData" )